In [1]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 313 kB/s 


# Punto 4

In [1]:
# Punto 4
from rdflib import Graph, Literal, BNode, URIRef, Namespace
# Creamos un grafo vacio
g = Graph()


In [2]:
# Parse a la ontologia
g.parse("computador.owl",format ="ttl")

<Graph identifier=Nf54f17468d054e07807b6cacae2f77b9 (<class 'rdflib.graph.Graph'>)>

In [3]:
# Hora de crear las consultas
# Primera consulta
# Lo que hace esta consulta es recoger todos los nombres de los individuos
consulta1 = """
PREFIX computador: <http://www.computador.org>
SELECT ?nombre
WHERE{
  ?individuo rdf:type owl:NamedIndividual.
  ?individuo rdfs:label ?nombre.
}
"""
for row in g.query(consulta1):
  print(row)

(rdflib.term.Literal('Asus_TUF_Gaming', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('ChromeBook_113180', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Computador_Gamer', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Computador_Hp_250', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Lenovo_IdeaCenter', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Lenovo_Ideapad_3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Macintosh_128K', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Victus_dlh007', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Victus_dlh008', datatype=rdflib.term.URIRef('http://www.w3.or

In [4]:
# Segunda consulta
# Se va a recoger los nombres de las clases cuya clase padre sea Parte_Computador
consulta2 = """
PREFIX computador: <http://www.computador.org>
SELECT ?nombre
WHERE{
  ?objeto rdf:type owl:Class;
    rdfs:label ?nombre;
    rdfs:subClassOf ?propiedad.
  ?propiedad rdfs:label ?nombrePropiedad.
  FILTER(?nombrePropiedad = "Parte_Computador").
}
"""
for row in g.query(consulta2):
  print(row)

(rdflib.term.Literal('Board', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Disco_Duro', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('GPU', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Nucleo_CPU', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Ram', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('Tipo_Refrigeracion', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)


# Punto 3

In [3]:
# Importamos la libreria SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST,JSON

# Indicamos el endpoint donde haremos la consulta
sparql = SPARQLWrapper("http://localhost:8890/sparql-auth")


# Damos las credenciales para hacer la consulta
sparql.setHTTPAuth(DIGEST)
sparql.setCredentials("dba", "dba")
sparql.setMethod(POST)

In [7]:
# Consulta e insertar con dbpedia
consulta = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/datatype/>
select distinct ?Marca ?Nombre
where {
  SERVICE <https://dbpedia.org/sparql/> {
    ?Marca dbo:product dbr:Hard_disk_drive .
    ?Marca rdfs:label ?Nombre .
    FILTER(LANG(?Nombre ) = "en")
}
}"""
insertar = """
PREFIX computador: <http://www.computador.org>
INSERT DATA{
"""
for row in g.query(consulta):
  insertar += "<" + str(row["Marca"]) + "> rdf:type owl:NamedIndividual. <" + str(row["Marca"]) + "> rdfs:label \""+str(row["Nombre"])+"\"."
insertar = insertar + "}"
sparql.setQuery(insertar)




In [6]:
# Consultar e insertar con wikidata
consulta = """
prefix wdt: <http://www.wikidata.org/prop/direct/>
prefix wd: <http://www.wikidata.org/entity/>
SELECT distinct ?parteComputador ?parteComputadorLabel WHERE {
  SERVICE <https://query.wikidata.org/sparql> {
    ?parteComputador wdt:P361 wd:Q68.
    ?parteComputador rdfs:label ?parteComputadorLabel.
    FILTER( LANG(?parteComputadorLabel) = "en" ).
  }
}

"""
insertar = """
PREFIX computador: <http://www.computador.org>
INSERT DATA{
"""
for row in g.query(consulta):
  consulta += "<"+ str(row["parteComputador"]) + "> rdfs:subClassOf computador:Parte_Computador. <" + str(row["parteComputador"]) + "> rdfs:label \""+str(row["parteComputadorLabel"])+"\"."
consulta += "}"
insertar = insertar + "}"
sparql.setQuery(insertar)